#### 1. Import pandas library

In [2]:
import pandas as pd

#### 2. Import users table:

In [5]:
users_table = pd.read_csv('users_table.csv')
users_table.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\r\n\r\n<p>...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN


#### 3. Rename Id column to userId

In [33]:
users_table.rename(columns={'Id':'userId'}, inplace=True)
users_table.head(20)

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\r\n\r\n<p>...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN
5,6,457,2010-07-19 19:04:07,Harlan,2014-08-07 19:49:44,http://www.harlan.harris.name,District of Columbia,<ul>\r\n<li>PhD in CS/AI/Machine Learning/Cogn...,114,47,0,46050,41.0,NaN
6,7,429,2010-07-19 19:04:37,Vince,2014-09-10 21:14:12,http://bioinformatics.ucdavis.edu,"Davis, CA",I'm a recent graduate of UC Davis in Economics...,56,20,0,49514,28.0,NaN
7,8,6764,2010-07-19 19:04:52,csgillespie,2014-09-09 21:15:08,http://www.mas.ncl.ac.uk/~ncsg3/,"Newcastle, United Kingdom",<p>I'm a statistics lecturer at Newcastle Univ...,1089,604,25,70002,36.0,NaN
8,10,121,2010-07-19 19:05:40,Pierre,2014-07-28 18:15:02,http://plindenbaum.blogspot.com,France,Bioinformatician\\nVirology\\nGenetics\\nBiolo...,20,2,0,23234,44.0,NaN
9,11,136,2010-07-19 19:06:02,wahalulu,2014-07-23 21:28:18,http://www.linkedin.com/in/marckvaisman,"Washington, DC","<p>data_stuff &lt;- paste('data', c('scientist...",10,10,0,47893,40.0,NaN


#### 4. Import posts table:

In [12]:
posts_table = pd.read_csv('posts_table.csv')
posts_table.head()

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [23]:
posts_table.rename(columns={'Id':'postId', 'OwnerUserId': 'userId'}, inplace=True)
posts_table.head()

,postID,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [32]:
users_columns = users_table[['userId', 'Reputation', 'Views', 'UpVotes','DownVotes']].copy()
posts_columns = posts_table[['postID', 'Score','userId', 'ViewCount', 'CommentCount']].copy()

users_columns.head()
#posts_columns.head()

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5


#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [41]:
merged_db = pd.merge(users_columns, posts_columns, on="userId")
merged_db.tail(50)

,userId,Reputation,Views,UpVotes,DownVotes,postID,Score,ViewCount,CommentCount
38912,26105,900,101,62,0,46599,2,NaN,1
38913,26105,900,101,62,0,46681,5,527.0,5
38914,26105,900,101,62,0,46766,2,NaN,2
38915,26105,900,101,62,0,47791,3,202.0,0
38916,26338,4800,515,1781,47,28238,0,NaN,0
38917,26338,4800,515,1781,47,28328,0,NaN,0
38918,26338,4800,515,1781,47,28978,0,NaN,0
38919,26338,4800,515,1781,47,38854,0,NaN,0
38920,26338,4800,515,1781,47,38877,0,NaN,0
38921,26338,4800,515,1781,47,38879,0,NaN,0


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [42]:
merged_db.isnull().sum()

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postID              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [52]:
# I think it has to be cleaned. The NaN values in ViewCount can't all be one values e.g. 0 
# because they also have upvote and downvote figures. Anything we input will make the data inaccurate

dropped_db = merged_db.dropna()
dropped_db.tail(50)

(15390, 9)

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [ ]:
#Not sure, maybe change ViewCount to an object?